Loading the InsuranceCosts table in our Database

In [ ]:
# Mount the capstone container (output)
from config import storageAccount
from config import storageContainer
from config import clientSecret
from config import clientid
mount_point = "/mnt/capstone-group2-data/datain"
    
    
configs = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs)

In [ ]:
df = spark.read.option("header", "true").csv("/mnt/capstone-group2-data/datain/cleandata/cleanHealthCosts/clean_insurance")

In [ ]:
#Age breakout table
age_df = df.select("age").distinct().sort('age')
age_df = age_df.withColumnRenamed('age', 'AgeLabel')
display(age_df)

In [ ]:
#Load into SQL Database
from config import server
from config import database
from config import user
from config import password
table = "dbo.Age"

age_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Sex breakout table
sex_df = df.select("sex").distinct()
sex_df = sex_df.withColumnRenamed('sex', 'SexLabel')
display(sex_df)

In [ ]:
#Load into SQL Database
table = "dbo.Sex"

appended.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Region breakout table
region_df = df.select("region").distinct()
region_df = region_df.withColumnRenamed('region', 'RegionLabel')
display(region_df)

In [ ]:
#Load into SQL Database
table = "dbo.Region"

region_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Smoker breakout table
smoker_df = df.select("smoker").distinct()
smoker_df = smoker_df.withColumnRenamed('smoker', 'SmokerLabel')
display(smoker_df)

In [ ]:
#Load into SQL Database
table = "dbo.Smoker"

smoker_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

In [ ]:
#Children breakout table
children_df = df.select("children").distinct().sort('children')
children_df = children_df.withColumnRenamed('children', 'ChildrenLabel')
display(children_df)

In [ ]:
#Load to SQL Database
table = "dbo.Children"

children_df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()

Make the InsuranceCharges table through joins of other tables

In [ ]:
table = "dbo.Sex"

#Read from server
sex_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
sex_df.show()

In [ ]:
df = df.join(sex_df, df.sex ==  sex_df.SexLabel, "left")

In [ ]:
table = "dbo.Age"

#Read from server
age_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
age_df.show()

In [ ]:
df = df.join(age_df, df.age ==  age_df.AgeLabel, "left")

In [ ]:
table = "dbo.Region"

#Read from server
region_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
region_df.show()

In [ ]:
df = df.join(region_df, df.region ==  region_df.RegionLabel, "left")

In [ ]:
table = "dbo.Smoker"

#Read from server
smoker_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
smoker_df.show()

In [ ]:
df = df.join(smoker_df, df.smoker ==  smoker_df.SmokerLabel, "left")

In [ ]:
table = "dbo.Children"

#Read from server
children_df = spark.read.format("jdbc") \
    .option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

#show the data loaded into dataframe
children_df.show()

In [ ]:
df = df.join(children_df, df.children ==  children_df.ChildrenLabel, "left")
display(df)

In [ ]:
df = df.drop("age", "children", "region", "sex", "smoker", "timestamp", "SexLabel", "AgeLabel", "RegionLabel", "SmokerLabel", "ChildrenLabel")

In [ ]:
df = df.withColumnRenamed('charges', 'ChargeValue')
df = df.withColumnRenamed('bmi', 'BMI')

In [ ]:
display(df)

In [ ]:
#Load to SQL Database
table = "dbo.InsuranceCharges"

df.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("append") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()